# MTH3302 : Méthodes probabilistes et statistiques pour l'I.A.

Jonathan Jalbert<br/>
Professeur adjoint au Département de mathématiques et de génie industriel<br/>
Polytechnique Montréal<br/>

# Devoir 3 - Régression logistique ( 15 points )

Le présent devoir concerne les notions de la régression logistique vues au Chapitre 3 du cours. 

## Devoir effectué par

NOM, PRÉNOM - MATRICULE

NOM, PRÉNOM - MARTICULE

**Déclaration d’intégrité relative au plagiat**

Nous certifions sur l’honneur :
- Que les résultats décrits dans ce rapport sont l’aboutissement de notre travail.
- Que nous sommes les auteurs de ce rapport.
- Que nous n'avons pas utilisé des sources ou résultats tiers sans clairement les citer et les référencer selon les règles bibliographiques préconisées.

Nous déclarons que ce travail ne peut être soupçonné de plagiat.


# Contexte : Classification des tumeurs

Une tumeur est un groupe de cellules anormales qui forment une masse. Les tumeurs se développent et se comportent différemment, selon qu’elles soient cancéreuses (malignes), non cancéreuses (bénignes). Le but du concours sera de prédire si une tumeur est bénigne (0) ou maligne (1) en fonction des caractéristiques suivantes récoltées par imagerie médicale : 

- radius : distance moyenne entre le centre de la tumeur et son périmètre (en mm) ;
- texture : écart-type des niveaux de gris représentant l'image de la tumeur ;
- perimeter : périmètre de la tumeur (en mm) ;
- area : superficie de la tumeur (en mm²) ;
- smoothness : variation locale normalisée en fonction du radius (indice de rugosité) ;
- compactness : perimeter^2 / area -1 (indice de compacité) ;
- symmetry : mesure de symétrie ; 
- fractal dimension : ("coastline approximation" - 1).

Les données sont issues de 569 images. Vous aurez à partitionner le jeu de données en ensemble d'entraînement et en ensemble de validation. Le code est fourni dans les premières cellules de code. La partition est fonction de vos numéros de matricules, alors vous avez tous des échantillons uniques.

# Consignes

- Le devoir doit être remis sur Moodle **avant 23h59 le lundi 22 février 2020.** Les retards ne seront pas tolérés car le solutionnaire sera publié dès mardi.
- Le devoir a une pondération de 7.5%.
- Écrivez vos noms et vos matricules dans l'espace prévu à cet effet de cette cellule et dans la cellule pour le partitionnement.
- Vous pouvez compléter le devoir en équipe de deux personnes. Dans ce cas, un seul fichier par équipe doit être remis. Pour vous aider à trouver un coéquipier, le canal de discussion *#trouver_un_coéquipier* a été créé sur Slack.
- Le devoir doit être complété dans le présent calepin Jupyter en ajoutant autant de cellules de texte et de code que nécessaires. Vous remettrez ce fichier en format *.ipynb*. Le nom du fichier doit être *matricule1_matricule2_devoir3.ipynb*.
- N'utilisez que les librairies introduites à la cellule suivante.
- Vous pouvez utiliser directement les fonction `auc()` et `rocplot()` fournies. 

## Chargement des libairies

In [ ]:
using CSV
using Combinatorics
using DataFrames
using Dates
using GLM
using Gadfly
using MLBase
using Random
using Statistics

## Écriture des matricules

Écrivez vos matricules ici. Si vous êtes seule, laissez le matricule2 à 0.

In [ ]:
matricule1 = 123456
matricule2 = 0

In [ ]:
"""
    auc(gt::Array{<:Real}, scores::Array{<:Real})

Compute the area under the ROC curve based on the ground truth `gt` and the success probability `scores`.

See also `roc()` of MLBase.
"""
function auc(gt::Array{<:Real},scores::Array{<:Real})

    # Compute the ROC curve for 100 equally spaced thresholds - see `roc()`
    r = roc(gt, scores, 0:.01:1)

    # Compute the true positive rate and false positive rate
    tpr = true_positive_rate.(r)
    fpr = false_positive_rate.(r)

    # Numerical computation of the area under the ROC curve
    p = sortperm(fpr)

    permute!(tpr,p)
    permute!(fpr,p)

    area = 0.0

    for i in 2:length(tpr)
        dx = fpr[i] - fpr[i-1]
        dy = tpr[i] - tpr[i-1]
        area += dx*tpr[i-1] + dx*dy/2
    end

    return area

end


"""
    rocplot(gt::Array{<:Real},scores::Array{<:Real})

Show the ROC curve corresponding to the ground truth `gt` and the success probability `scores`.

The curve is computed for 100 equally spaced thresholds.
"""
function rocplot(gt::Array{<:Real},scores::Array{<:Real})

    # Compute the ROC curve for 100 equally spaced thresholds - see `roc()`
    r = roc(gt, scores, 0:.01:1)

    # Compute the true positive rate and false positive rate
    tpr = true_positive_rate.(r)
    fpr = false_positive_rate.(r)

    return plot(x=fpr, y=tpr, Geom.line, Geom.abline(color="red", style=:dash),
        Guide.xlabel("False Positive Rate"), Guide.ylabel("True Positive Rate"))

end

## Chargement et partitionnement des données

#### Chargement des données

In [ ]:
data = CSV.read("data.csv", DataFrame)
first(data, 10)

#### Partitionnement en ensemble d'entraînement et de validation

Le partitionnement est le suivant :
- ensemble d'entraînement : 80%
- ensemble d'entraînement : 20%

In [ ]:
n = Int(round(nrow(data)*.8))

Random.seed!(matricule1+matricule2)

train_id = sample(1:nrow(data), n, replace=false, ordered=true)
valid_id = setdiff(1:nrow(data), train_id)

train = data[train_id, :]
first(train,10)

In [ ]:
valid = data[valid_id, :]
first(valid,10)

## Exercice 1 - Analyse exploratoire sommaire (3 points)

On s'intéresse ici à illustrer le lien entre le caractère bénin (:diagnosis = 0) ou malin (:diagnosis = 1) des tumeurs en fonction du rayon de celles-ci. **N'ajustez pas de modèle statistique pour répondre aux questions de l'exercice 1.**

Cet exercice ne constitue que la première étape d'une analyse exploratoire sommaire. Je vous encourage à poursuivre l'analyse exploratoire pour mieux cerner le problème général. 

#### (a) Vérifiez à l'aide d'un graphique approprié si le rayon de la tumeur semble influencer le diganostic.

Choisissez le graphique qui vous semble le plus éloquent. Imaginez que vous devez le présentez à un groupe de médecins qui ne connaissent rien à la statistique et à l'apprentissage machine.

#### (b) Parmi les tumeurs dont le rayon excède 12 mm, calculez la proportion empirique des tumeurs malignes.

#### (c) Seriez-vous en mesure de calculez le rayon pour lequel 95% des tumeurs sont malignes ?

Si oui, calculez-le. Sinon, expliquez pourquoi c'est difficile.

## Exercice 2 - Modélisation du diagnostic en fonction du rayon (8 points)

#### a) Estimation des paramètres

En utilisant comme seule variable explicative le rayon de la tumeur, estimez les paramètres avec les données de l'ensemble d'entraînement du modèle de régression logistique en utilisant le fonction de lien logit. Donnez explicitement β̂₀ et β̂₁.

**Suggestion :** Utilisez la fonction `glm()` de la librairie *GLM.jl*.

#### b) Si une tumeur possède un rayon de 12 mm, quelle est l'estimation de la probabilité quelle soit maligne ?

#### c) Quelle est le rayon pour lequel on est sûr à 80% que la tumeur est maligne ?

Expliquez bien les détails de votre calcul.

#### d) Évaluation de la qualité du modèle

Tracez la courbe ROC du modèle et calculez l'aire sous la courbe. Donnez une interprétation pratique de ces mesures.

#### e) Règle de décision

Laquelle des règles suivantes est la plus appropriée pour prédire la malignité d'une tumeur :

- Règle 1 : si le rayon dépasse 12 *mm*.
- Règle 2 : si le rayon dépasse 15 *mm*.
- Règle 3 : si le rayon dépasse 17.2 *mm*.
- Règle 4 : si le rayon dépasse 19.5 *mm*.

Justifiez bien quantitativement votre réponse. Utilisez les mesures de qualité qui vous semblent les plus appropriées.

#### f) Justification de la méthodologie

Un médecin vous regarde incrédule faire la présentation de votre modèle et vous demande à quoi ça sert de faire aussi compliqué. D'après lui, après avoir vu le graphique présenté à l'analyse exploratoire, il est en mesure d'estimer le rayon à partir de lequel les tumeur peuvent être étiquetés comme malignes. À l'oeil, il estime le seuil 15 *mm*. Comment pouvez justifier l'utilisation de votre approche ? 

**Note :** C'est une question typique que vous vous ferez certainement poser !

## Exercice 3 - Prédiction des diagnostics (4 points)

Identifiez les variables permettant de mieux prédire les diagnostics. Vous devez obligatoirement utilisée la régression logistique mais les détails de la méthodologie sont laissés à votre discrétion. Vous pouvez utiliser comme bon vous semble la courbe ROC, l'échantillon d'entraînement, l'échantillon d'entraînement, etc.

**Note :** Vous ne devez ici qu'identifier les variables explicatives les plus importantes pour la prédiction des diagnositics. Vous ne devez pas spécifier la règle de décision optimale.